# Laboratório: Q-learning no Mundo Grade

Este notebook explora o algoritmo **Q-learning** em um ambiente do tipo *Grid World*.
O objetivo é entender como a definição de recompensas, a estocasticidade das ações e os parâmetros de aprendizado afetam a política aprendida.

O ambiente segue a convenção:
- Origem `[1,1]` no canto inferior esquerdo.
- `x` representa colunas (esquerda → direita).
- `y` representa linhas (baixo → cima).

In [2]:
# Código base: Q-learning em um Mundo Grade 3x4
import random
from collections import defaultdict

C, R = 4, 3
START = (1, 1)
OBSTACLE = (2, 2)
TERM_POS = (4, 3)
TERM_NEG = (4, 2)

ACTIONS = {
    'U': (0, +1),
    'D': (0, -1),
    'L': (-1, 0),
    'R': (+1, 0),
}

STEP_COST = -0.04
INVALID_COST = -0.10

def valid_state(s):
    x, y = s
    return 1 <= x <= C and 1 <= y <= R and s != OBSTACLE

def is_terminal(s):
    return s in (TERM_POS, TERM_NEG)

def reward_for(s):
    if s == TERM_POS: return +1.0
    if s == TERM_NEG: return -1.0
    return STEP_COST

def step(s, a):
    if is_terminal(s):
        return s, 0.0, True
    dx, dy = ACTIONS[a]
    nxt = (s[0] + dx, s[1] + dy)
    if not valid_state(nxt):
        return s, INVALID_COST, False
    r = reward_for(nxt)
    done = is_terminal(nxt)
    return nxt, r, done

Q = defaultdict(float)

def epsilon_greedy(s, eps):
    if random.random() < eps:
        return random.choice(list(ACTIONS.keys()))
    best_a, best_q = None, float('-inf')
    for a in ['U','D','L','R']:
        q = Q[(s, a)]
        if q > best_q:
            best_q, best_a = q, a
    return best_a

alpha = 0.1
gamma = 0.99
episodes = 4000

def lin_decay(t, T, a, b):
    return a + (b - a) * (t / max(1, T - 1))

for ep in range(episodes):
    s = START
    eps = lin_decay(ep, episodes, 0.9, 0.05)
    while True:
        a = epsilon_greedy(s, eps)
        s2, r, done = step(s, a)
        if done:
            target = r
        else:
            max_q_next = max(Q[(s2, a2)] for a2 in ACTIONS)
            target = r + gamma * max_q_next
        Q[(s, a)] += alpha * (target - Q[(s, a)])
        s = s2
        if done:
            break

def best_action(s):
    if not valid_state(s): return 'X'
    if s == TERM_POS: return '+1'
    if s == TERM_NEG: return '-1'
    a = max(ACTIONS, key=lambda act: Q[(s, act)])
    return {'U':'↑','D':'↓','L':'←','R':'→'}[a]

print("Optimal policy (origin=bottom-left; display: top→bottom)")
for y in range(R, 0, -1):        # topo (y=R) primeiro
    row = []
    for x in range(1, C + 1):
        s = (x, y)
        if s == OBSTACLE:
            row.append(" X ")
        elif s == TERM_POS:
            row.append(" +1")
        elif s == TERM_NEG:
            row.append(" -1")
        else:
            row.append(f"{best_action(s):^3}")
    print(" ".join(row))


Optimal policy (origin=bottom-left; display: top→bottom)
 →   →   →   +1
 ↑   X   ↑   -1
 ↑   →   ↑   ← 


# Exercícios

### **Exercício 1 – Alterar recompensas**

Modifique a função `reward_for()` para investigar como a política muda:

1. Aumente o custo de passo (`STEP_COST`) para `-0.2`.
2. Diminua o custo de bater na parede (`INVALID_COST`) para `-0.01`.
3. Compare as políticas resultantes e descreva o comportamento qualitativamente.

### **Exercício 2 – Introduzir estocasticidade**

Na função `step()`, faça com que cada ação tenha 80% de chance de ser executada corretamente e 20% de chance de virar à esquerda ou direita aleatoriamente.
Verifique se o agente ainda encontra o terminal positivo.


### **Exercício 3 – Ajustar hiperparâmetros**

Varie:

* taxa de aprendizado `alpha ∈ {0.1, 0.5}`
* fator de desconto `gamma ∈ {0.8, 0.99}`
* número de episódios `episodes ∈ {1000, 10000}`

Observe o impacto no tempo de convergência e na estabilidade da política final.

### **Exercício 4 – Visualizar convergência**

Adicione um gráfico da soma de recompensas por episódio.
Use `matplotlib` e mostre a curva de aprendizado do agente.

### **Exercício 5 – Política exploratória**

Implemente um decaimento **exponencial** de epsilon em vez do linear usado (`lin_decay`).
Compare o número de episódios necessários até a política estabilizar.

### **Exercício 6 – Novo terminal**

Adicione um novo estado terminal em `[1,3]` com recompensa `+0.5`.
Treine novamente e observe se o agente passa a preferir esse novo caminho.

## Exercício 1 – Alterar recompensas

In [ ]:
# TODO: altere STEP_COST e INVALID_COST e reexecute o treinamento

## Exercício 2 – Introduzir estocasticidade

In [ ]:
# TODO: introduza ruído de transição (usar random.random() < 0.8)

## Exercício 3 – Ajustar hiperparâmetros

In [ ]:
# TODO: experimente diferentes combinações de alpha, gamma e episodes

## Exercício 4 – Visualizar convergência

In [ ]:
# TODO: registre a soma das recompensas e plote a curva de aprendizado

## Exercício 5 – Política exploratória

In [ ]:
# TODO: substitua lin_decay por decaimento exponencial

## Exercício 6 – Novo terminal

In [ ]:
# TODO: inclua o novo terminal na função reward_for()